In [ ]:
%pip install langchain langchain-openai tiktoken
%pip install faiss-cpu # for local vector DB
%pip install beautifulsoup4

In [ ]:
import os
import os
import json
# we load the config file that has the same schema as project's sample-config.json but named config.json
# replace values for your specific api keys in the config.json file.
home_dir = os.path.expanduser("~")
cfgFile = os.path.join(home_dir, ".langchain", "config.json")
configData = json.load(open(cfgFile, "r"))

#print(configData["openAI"]["apiKey"])
# read and set all environment variables
os.environ["OPENAI_API_KEY"] = configData["openAI"]["apiKey"]
os.environ["OLLAMA_HOST"] = "127.0.0.1" # we are running the ollama server locally
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# now initialize the OpenAI Chat model:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

# from langchain_community.llms import Ollama
# llm = Ollama(model="llama2")

In [ ]:
llm.invoke("how can langsmith help with testing?")

Prompt templates are used to convert raw user input to a better input to the LLM.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

We can now combine these into a simple LLM chain:

In [ ]:
chain = prompt | llm 

We can now invoke it and ask the same question. \
It still won't know the answer, but it \
should respond in a more proper tone for a technical writer!

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

The output of a ChatModel (and therefore, of this chain) is a message. \
However, it's often much more convenient to work with strings. \
Let's add a simple output parser to convert the chat message to a string.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

We can now add this to the previous chain:

In [ ]:
chain = prompt | llm | output_parser

We can now invoke it and ask the same question.
The answer will now be a string (rather than a ChatMessage).

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

Retrieval Chain
A Retriever can be backed by anything - a SQL table, the internet, etc.\
But in this instance we will populate a vector store and use that as a retriever. 

In order to do this, we will use the WebBaseLoader. \
This requires installing BeautifulSoup:

In [ ]:
# now use webBaseLoader
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

Next, we need to index it into a vectorstore. 
This requires a few components, namely an embedding model and a vectorstore.
For embedding models, we can use OpenAI or local models.

For local, ensure you have Ollama running (same set up as with the LLM).


In [ ]:
# for OpanAI embeddings we can use:
# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings()
# We'll need a local instance of Ollam running. Ensur ethis is running first before invoking calls to the embeddings
# we can use a local docker image, to start a locall instance of Ollam in docker:
# docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

Now, we can use this embedding model to ingest documents into a vectorstore. \
We will use a simple local vectorstore, FAISS, for simplicity's sake.\
First we need to install the required packages for that:

In [ ]:
# Now build the index:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
#print(vector)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. \
This chain will take an incoming question, look up relevant documents, \
then pass those documents along with the original question into an LLM and ask \
it to answer the original question.
First, let's set up the chain that takes a question and the \
retrieved documents and generates an answer.

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

we want the documents to first come from the retriever we just set up. \
That way, for a given question we can use the retriever to dynamically \
select the most relevant documents and pass those in.

In [ ]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

We can now invoke this retrieval chain. \
This returns a dictionary - the response from the LLM is in the answer key

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

In [ ]:
print(llm.model_name)

In [ ]:
# Now update the code to create a Conversation Retrieval Chain
# n order to update retrieval, we will create a new chain. 
# This chain will take in the most recent input (input) and the 
# conversation history (chat_history) and use an LLM to generate a search query.

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
# We can test this out by passing in an instance where the user is asking a follow up question.
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

you should see that this returns documents about testing in LangSmith. \
This is because the LLM generated a new query, combining \
the chat history with the follow up question.

Now that we have this new retriever, we can create a new \
chain to continue the conversation with these retrieved documents in mind.

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
# We can now test this out end-to-end:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

We can see that this gives a coherent answer..!\
We've successfully turned our retrieval chain into a chatbot!